In [12]:
import pandas as pd
import numpy as np
import pickle

dataframes = pd.read_pickle('recom_data/user_category_dummy.pkl')

df_counts = dataframes['view_counts']
df_users = dataframes['users']

with open('recom_data/category_dict.pkl', 'rb') as handle:
    category_dict = pickle.load(handle)

item_similarity = pd.read_pickle('recom_data/item_similarity.pkl')

In [13]:
category_dict[855] = '유아동 도서'
category_dict[1071] = '교육 도서'
category_dict[1072] = '학습 도서'

In [14]:
view_matrix = df_counts.pivot(index='user', columns='category', values='count')

In [3]:
class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        """
            K: 잠재요인(latent factor)의 수
            alpha: 학습률
            beta: 정규화 계수
            iterations: SGD의 계산을 할 때의 반복 횟수
            verbose: SGD의 중간 학습과정 출력 여부
        """
        self.R = np.array(ratings)
        
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose
        
    def rmse(self):
        """
            현재의 P행렬과 Q행렬을 가지고 Root Mean Squared Error(RMSE)를 계산
        """
        xs, ys = self.R.nonzero()
        # R에서 평점이 있는 (0이 아닌) 요소의 인덱스를 저장
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            # 평점이 존재하는 요소 (사용자 x, 아이템 y) 각각에 대해 아래 코드를 실행
            prediction = self.get_prediction(x, y)
            # 사용자 x, 아이템 y에 대한 평점 예측치를 계산
            self.predictions.append(prediction)
            # 예측값을 리스트에 추가
            self.errors.append(self.R[x, y] - prediction)
            # 실제값과 예측값의 차이를 계산해서 오차값 리스트에 추가
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        # errors를 사용해서 RMSE를 계산
        return np.sqrt(np.mean(self.errors**2))
    
    def train(self):
        """
            정해진 횟수만큼 반복하며 P, Q, bu, bd 값을 업데이트하는 함수
        """
        # Initializing user-feature and movie-feature matrix
        # P, Q 행렬을 임의의 값으로 채움
        # 평균이 0, 표준편차가 10/K인 정규분포를 갖는 난수로 초기화
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initializing the bias terms
        # 사용자 평가 경향(bu)를 0으로 초기화 / 크기는 사용자 수 (num_users)
        # 아이템 평가 경향(bd)를 0으로 초기화 / 크기는 아이템 수 (num_items)
        # 전체 평균 b를 구해서 저장
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])
        
        # List of training samples
        # 평점 행렬에서 값이 존재하는 요소의 인덱스들을 가져옴
        # SGD를 적용할 대상 (평점이 있는 요소의 인덱스와 평점)을 리스트로 저장
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]
        
        # Stochastic gardient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            # samples를 임의로 섞음 => 어디서 시작하는지에 따라 수렴의 속도가 달라짐
            # 매 반복마다 다양한 시작점에서 시작하기!
            self.sgd()
            # sgd를 실행 / P, Q, bu, bd가 업데이트 됨
            rmse = self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("반복: {0}; Train RMSE = {1:0.4f}".format(i+1, rmse))
        return training_process
    
    def get_prediction (self, i, j):
        """
            Rating prediction for user i and item j
            사용자 i의 아이템 j에 대한 평점을 예측하는 함수
        """
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
        
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])
    
    def full_prediction(self):
        return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)

    def sgd(self):
        """
            Stochastic gardient descent to get optimized P and Q matrix
        """
        for i, j, r in self.samples:
            # samples에 있는 각 사용자-아이템-평점 세트에 대해서 SGD를 적용
            prediction = self.get_prediction(i, j)
            # 사용자 i, 아이템 j에 대한 평점 예측치를 구함
            e = (r - prediction)
            # 실제 평점 r과 비교해서 오차를 구함
            
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])
            # 사용자 평가 경향, 아이템 평가 경향 업데이트
            
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])

In [4]:
view_matrix = df_counts.pivot(index='user', columns='category', values='count')
R_temp = view_matrix.fillna(0)
mf = MF(R_temp, K=100, alpha=0.01, beta=0.2, iterations=100, verbose=True)
# 인자 40개 / 반복 50회 / 학습률 0.001 / 정규화 계수 0.02로 모델을 학습합니다.
mf.train()
print(' ')

반복: 10; Train RMSE = 0.8524
반복: 20; Train RMSE = 0.8518
반복: 30; Train RMSE = 0.8510
반복: 40; Train RMSE = 0.8473
반복: 50; Train RMSE = 0.8388
반복: 60; Train RMSE = 0.8314
반복: 70; Train RMSE = 0.8273
반복: 80; Train RMSE = 0.8248
반복: 90; Train RMSE = 0.8228
반복: 100; Train RMSE = 0.8213
 


In [5]:
print(mf.full_prediction())
# 전체 user-category의 예측값들입니다.

[[2.00167827 2.09482795 2.04042982 ... 2.53493103 2.13708008 2.15841162]
 [1.73322405 1.77027175 1.78761272 ... 1.91051032 1.59569422 1.62750269]
 [1.98729984 2.01449734 2.04809963 ... 2.09834859 1.80355931 1.83522653]
 ...
 [2.20382599 2.21004035 2.27720565 ... 2.18366077 1.92753196 1.95949675]
 [2.20944556 2.21613093 2.28278798 ... 2.192187   1.93544577 1.96727523]
 [2.20484414 2.21121158 2.27818863 ... 2.18560839 1.92925802 1.96119338]]


In [6]:
testers = [
    {'user_id': 90001,'gender': '남성','age': 17}, {'user_id': 90081,'gender': '남성','age': 22}, {'user_id': 90161,'gender': '남성','age': 32}, {'user_id': 90221,'gender': '남성','age': 42}, {'user_id': 90281,'gender': '남성','age': 52}, {'user_id': 90341,'gender': '남성','age': 62}, {'user_id': 90401,'gender': '남성','age': 72},
    {'user_id': 90501,'gender': '여성','age': 12}, {'user_id': 90521,'gender': '여성','age': 22}, {'user_id': 90541,'gender': '여성','age': 32}, {'user_id': 90561,'gender': '여성','age': 42}, {'user_id': 90581,'gender': '여성','age': 52}, {'user_id': 90601,'gender': '여성','age': 62}, {'user_id': 90621,'gender': '여성','age': 72}
]

# 각각의 그룹에서 한명씩 유저를 데려오고... 그 유저의 추천 카테고리를 출력해봤습니다.
# 그룹화가 아니라 개인화 추천입니다!!! 같은 그룹이어도 다른 추천 결과가 나옵니다.
for tester in testers:
    print(tester['gender'], tester['age'])
    temp_data = dict()
    
    # 전체 카테고리에 대해 예측값을 계산합니다.
    for category in R_temp:
        temp_data[category] = mf.get_one_prediction(tester['user_id'], category)

    # 예측값을 기준으로 정렬합니다.
    recomm_df = pd.DataFrame.from_dict([temp_data]).T.reset_index().rename(columns={'index': 'category', 0: 'pred'})
    temp_recomm_df = recomm_df.sort_values(by='pred', ascending=False)
    temp_recomm_df['category'] = temp_recomm_df['category'].map(lambda x: category_dict[x])
    print(temp_recomm_df)

남성 17
      category      pred
420      기타 문구  2.534931
419      사무 용품  2.423824
418      문구 용품  2.384112
3      여성 언더웨어  2.355415
412      기타 도서  2.323497
..         ...       ...
238  기타 관상어 용품  1.413329
264   기타 세탁 용품  1.402660
248  기타 자동차 용품  1.395205
247      안전 용품  1.391772
243        장난감  1.391548

[423 rows x 2 columns]
남성 22
      category      pred
327         인형  2.573523
320     임상부 용품  2.569935
3      여성 언더웨어  2.540266
8           기타  2.509890
319      수유 용품  2.485522
..         ...       ...
279     세제/세정제  1.671982
247      안전 용품  1.665354
243        장난감  1.662246
264   기타 세탁 용품  1.658584
248  기타 자동차 용품  1.657098

[423 rows x 2 columns]
남성 32
     category      pred
327        인형  2.773409
320    임상부 용품  2.718565
298     아기 간식  2.706104
319     수유 용품  2.662716
300       물티슈  2.651828
..        ...       ...
230        목공  1.724759
234  프라모델/피규어  1.719485
405     학습/교육  1.712060
235    기타 수집품  1.662006
410        잡지  1.644596

[423 rows x 2 columns]
남성 42
      category  

In [7]:
# 아래는 테스트중인 코드입니다. (svd 알고리즘) / https://www.fun-coding.org/recommend_basic6.html

In [15]:
view_matrix = view_matrix.fillna(0)
matrix_values = view_matrix.values
user_view_mean = np.mean(matrix_values, axis=1)

matrix_user_mean = matrix_values - user_view_mean.reshape(-1, 1)

In [16]:
from scipy.sparse.linalg import svds

user, sigma, vt = svds(matrix_user_mean, k = 80)
sigma = np.diag(sigma)

svd_user_prediction = np.dot(np.dot(user, sigma), vt) + user_view_mean.reshape(-1, 1)

In [17]:
test = list(category_dict.items())
test.sort(key=lambda x:x[0])
test2 = [x[1] for x in test]

results = []
for i in range(0, 1140, 20):
    result = svd_user_prediction[i]
    result_df = pd.DataFrame(result, columns=['pred'])
    result_df['category'] = result_df.index
    result_df['category'] = result_df['category'].apply(lambda x: test2[x])
    result_df = result_df.sort_values(by='pred', ascending=False)
    results.append([df_users.loc[i]['user'], df_users.loc[i]['gender'], df_users.loc[i]['age'], result_df])

results.sort(key=lambda x: x[0])
for result in results:
    print(result[0], result[1], result[2])
    print(result[3])

90001 M 10
         pred     category
360  3.464397         배드민턴
329  3.422307          수영복
343  3.302728          야구공
328  3.222383          수영모
330  3.081063     기타 수영 용품
..        ...          ...
370 -0.308548        당구 용품
368 -0.357282        무술 용품
1   -0.382604        여성 하의
365 -0.395043  기타 스포츠 액세서리
420 -0.459785        기타 문구

[423 rows x 2 columns]
90021 M 10
         pred category
5    3.352858    남성 하의
119  3.219918   스마트 밴드
23   2.907891      단지갑
45   2.809398    남성 장갑
16   2.547900   남성 운동화
..        ...      ...
411 -0.389565     전공서적
10  -0.426326  여성 캐주얼화
420 -0.482190    기타 문구
147 -0.505034      Wii
44  -0.548618    여성 장갑

[423 rows x 2 columns]
90041 M 10
         pred     category
375  4.230342     기타 등산 용품
377  2.879508        스키 장비
382  2.641029    기타 자전거 용품
376  2.474734      스노보드 장비
378  2.410283  기타 스키/보드 용품
..        ...          ...
67  -0.449633       기타 선케어
1   -0.459524        여성 하의
400 -0.511007         요가매트
20  -0.607327     기타 신발 용품
15  -0.627142      남성 

[423 rows x 2 columns]
90821 F 50
         pred   category
381  4.365503   자전거 안전용품
374  4.090015      등산 잡화
372  3.601293      등산 의류
384  3.340642     낚시 소모품
382  3.274620  기타 자전거 용품
..        ...        ...
404 -0.437433      교육 도서
391 -0.476681         아대
389 -0.514313        볼링공
390 -0.519583        볼링화
379 -0.604784        자전거

[423 rows x 2 columns]
90841 F 60
         pred   category
171  3.427182   기타 영상 가전
213  3.239145  기타 DIY 자재
203  3.045895     아동 서랍장
188  2.848629         장롱
166  2.727490       요리가전
..        ...        ...
56  -0.439266     남성용 시계
28  -0.440491   헤어밴드/핀/끈
25  -0.489689     남성용 가방
126 -0.621872         기타
50  -0.669365      여성 양말

[423 rows x 2 columns]
90861 F 60
         pred   category
213  4.437935  기타 DIY 자재
228  4.254305         수예
224  4.005393         액자
221  3.930068       블라인드
173  3.884551       의류관리
..        ...        ...
138 -0.391277    기타 주변기기
422 -0.403134         기타
0   -0.416152      여성 상의
14  -0.452317      남성 구두
123 -0.504418    PC 액